# 微分可能な MatterSim のデモ

このノートブックでは、MatterSim を PyTorch の autograd と統合し、入力（原子種、格子、座標）に対して勾配を計算する方法を示します。

## 概要

- **目的**: MatterSim を損失関数の一部として使用し、入力を最適化する
- **新しい API**: `DifferentiableMatterSimCalculator`
- **主な機能**: 原子種・格子・座標への勾配計算

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from ase.build import bulk
import sys

sys.path.append('../../')  # Adjust the path as needed to locate mattersim
import mattersim
from mattersim.forcefield.differentiable_potential import DifferentiableMatterSimCalculator

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

ModuleNotFoundError: No module named 'mattersim.forcefield.differentiable_potential'

## 1. 基本的な使い方：勾配の計算

まず、Diamond Si 構造を作成し、座標に対する勾配を計算します。

In [ ]:
# Diamond Si 構造を作成
si = bulk("Si", "diamond", a=5.43)
print(f"原子数: {len(si)}")
print(f"座標:\n{si.get_positions()}")
print(f"格子:\n{si.cell.array}")

In [ ]:
# 微分可能な Calculator を初期化
device = "cpu"  # または "cuda"
calc = DifferentiableMatterSimCalculator(device=device)
print("Calculator が初期化されました")

In [ ]:
# 座標を tensor として取得し、requires_grad を設定
positions = torch.tensor(
    si.get_positions(), dtype=torch.float32, device=device, requires_grad=True
)

print(f"座標 tensor の形状: {positions.shape}")
print(f"requires_grad: {positions.requires_grad}")

In [ ]:
# エネルギーを計算
output = calc.forward(si, positions=positions, include_forces=False)
energy = output["total_energy"]

print(f"エネルギー: {energy.item():.6f} eV")
print(f"エネルギー tensor の形状: {energy.shape}")
print(f"requires_grad: {energy.requires_grad}")

In [ ]:
# backward で勾配を計算
energy.backward()

print(f"座標の勾配形状: {positions.grad.shape}")
print(f"座標の勾配:\n{positions.grad}")
print(f"勾配のノルム: {torch.norm(positions.grad).item():.6e}")

✅ **結果**: 座標に対する勾配が正常に計算されました！

## 2. 構造最適化

ランダムに摂動を加えた構造を、PyTorch の optimizer で最適化します。

In [ ]:
# 新しい構造を作成（摂動を加える）
si_perturbed = bulk("Si", "diamond", a=5.43)
positions_opt = torch.tensor(si_perturbed.get_positions(), dtype=torch.float32, device=device)

# ランダムに摂動
torch.manual_seed(42)  # 再現性のため
perturbation = torch.randn_like(positions_opt) * 0.05  # 0.05 Å の摂動
positions_opt = positions_opt + perturbation
positions_opt.requires_grad_(True)

print(f"摂動のノルム: {torch.norm(perturbation).item():.6f} Å")
print(f"摂動後の座標:\n{positions_opt}")

In [ ]:
# Calculator を初期化（構造が変わったので再初期化）
calc_opt = DifferentiableMatterSimCalculator(device=device)

# Adam オプティマイザ
optimizer = torch.optim.Adam([positions_opt], lr=0.02)

# 最適化の履歴を記録
energy_history = []
grad_norm_history = []

print("最適化を開始します...")
print("ステップ | エネルギー (eV) | 勾配ノルム")
print("-" * 50)

num_steps = 30
for step in range(num_steps):
    optimizer.zero_grad()
    
    # エネルギーを計算
    output = calc_opt.forward(si_perturbed, positions=positions_opt, include_forces=False)
    energy = output["total_energy"]
    
    # backward
    energy.backward()
    
    # 履歴を記録
    energy_history.append(energy.item())
    grad_norm = torch.norm(positions_opt.grad).item()
    grad_norm_history.append(grad_norm)
    
    # 最適化ステップ
    optimizer.step()
    
    if step % 5 == 0 or step == num_steps - 1:
        print(f"{step:7d} | {energy.item():14.6f} | {grad_norm:.6e}")

print(f"\n初期エネルギー: {energy_history[0]:.6f} eV")
print(f"最終エネルギー: {energy_history[-1]:.6f} eV")
print(f"エネルギー減少: {energy_history[0] - energy_history[-1]:.6f} eV")

In [ ]:
# 最適化の履歴をプロット
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# エネルギーの推移
ax1.plot(energy_history, 'b-', linewidth=2)
ax1.set_xlabel('最適化ステップ', fontsize=12)
ax1.set_ylabel('エネルギー (eV)', fontsize=12)
ax1.set_title('エネルギーの推移', fontsize=14)
ax1.grid(True, alpha=0.3)

# 勾配ノルムの推移
ax2.semilogy(grad_norm_history, 'r-', linewidth=2)
ax2.set_xlabel('最適化ステップ', fontsize=12)
ax2.set_ylabel('勾配ノルム', fontsize=12)
ax2.set_title('勾配ノルムの推移', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

✅ **結果**: エネルギーが減少し、最適化が正常に動作しました！

## 3. 格子定数の最適化

格子定数を少しずらした構造から、正しい格子定数を最適化で求めます。

In [ ]:
# 格子定数を少しずらした構造
a_initial = 5.5  # 正しい値は 5.43
si_lattice = bulk("Si", "diamond", a=a_initial)

print(f"初期格子定数: a = {a_initial:.3f} Å")
print(f"正解の格子定数: a = 5.43 Å")
print(f"初期格子行列:\n{si_lattice.cell.array}")

In [ ]:
# Calculator を初期化
calc_lattice = DifferentiableMatterSimCalculator(device=device)

# 格子を tensor として取得
lattice_opt = torch.tensor(si_lattice.cell.array, dtype=torch.float32, device=device)
lattice_opt.requires_grad_(True)

# SGD オプティマイザ（格子は学習率を小さくする）
optimizer_lattice = torch.optim.SGD([lattice_opt], lr=0.01)

# 最適化の履歴
energy_lattice_history = []
lattice_a_history = []

print("格子定数の最適化を開始します...")
print("ステップ | エネルギー (eV) | 格子定数 a (Å)")
print("-" * 50)

num_steps_lattice = 40
for step in range(num_steps_lattice):
    optimizer_lattice.zero_grad()
    
    # エネルギーを計算
    output = calc_lattice.forward(si_lattice, lattice=lattice_opt, include_forces=False)
    energy = output["total_energy"]
    
    # backward
    energy.backward()
    
    # 最適化ステップ
    optimizer_lattice.step()
    
    # 現在の格子定数を記録
    current_a = lattice_opt[0, 0].item()
    energy_lattice_history.append(energy.item())
    lattice_a_history.append(current_a)
    
    if step % 5 == 0 or step == num_steps_lattice - 1:
        print(f"{step:7d} | {energy.item():14.6f} | {current_a:.6f}")

final_a = lattice_opt[0, 0].item()
print(f"\n初期格子定数: a = {a_initial:.6f} Å")
print(f"最終格子定数: a = {final_a:.6f} Å")
print(f"正解値: a = 5.43 Å")
print(f"誤差: {abs(final_a - 5.43):.6f} Å")

In [ ]:
# 格子最適化の履歴をプロット
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# エネルギーの推移
ax1.plot(energy_lattice_history, 'b-', linewidth=2)
ax1.set_xlabel('最適化ステップ', fontsize=12)
ax1.set_ylabel('エネルギー (eV)', fontsize=12)
ax1.set_title('エネルギーの推移（格子最適化）', fontsize=14)
ax1.grid(True, alpha=0.3)

# 格子定数の推移
ax2.plot(lattice_a_history, 'g-', linewidth=2, label='最適化された値')
ax2.axhline(y=5.43, color='r', linestyle='--', linewidth=2, label='正解値')
ax2.set_xlabel('最適化ステップ', fontsize=12)
ax2.set_ylabel('格子定数 a (Å)', fontsize=12)
ax2.set_title('格子定数の推移', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 複数パラメータの同時最適化

座標と格子を同時に最適化します。

In [ ]:
# 新しい構造（座標と格子の両方を摂動）
si_joint = bulk("Si", "diamond", a=5.5)

# 座標を摂動
positions_joint = torch.tensor(si_joint.get_positions(), dtype=torch.float32, device=device)
torch.manual_seed(123)
positions_joint = positions_joint + torch.randn_like(positions_joint) * 0.03
positions_joint.requires_grad_(True)

# 格子も tensor に
lattice_joint = torch.tensor(si_joint.cell.array, dtype=torch.float32, device=device)
lattice_joint.requires_grad_(True)

print(f"初期格子定数: a = {lattice_joint[0, 0].item():.6f} Å")
print(f"座標の摂動ノルム: {torch.norm(positions_joint - torch.tensor(si_joint.get_positions(), device=device)).item():.6f} Å")

In [ ]:
# Calculator を初期化
calc_joint = DifferentiableMatterSimCalculator(device=device)

# オプティマイザ（両方のパラメータを含む）
optimizer_joint = torch.optim.Adam([
    {'params': [positions_joint], 'lr': 0.02},
    {'params': [lattice_joint], 'lr': 0.005},  # 格子は学習率を小さく
])

# 最適化の履歴
energy_joint_history = []
lattice_a_joint_history = []

print("同時最適化を開始します...")
print("ステップ | エネルギー (eV) | 格子定数 a (Å)")
print("-" * 50)

num_steps_joint = 30
for step in range(num_steps_joint):
    optimizer_joint.zero_grad()
    
    # エネルギーを計算
    output = calc_joint.forward(
        si_joint, 
        positions=positions_joint, 
        lattice=lattice_joint, 
        include_forces=False
    )
    energy = output["total_energy"]
    
    # backward
    energy.backward()
    
    # 最適化ステップ
    optimizer_joint.step()
    
    # 履歴を記録
    current_a = lattice_joint[0, 0].item()
    energy_joint_history.append(energy.item())
    lattice_a_joint_history.append(current_a)
    
    if step % 5 == 0 or step == num_steps_joint - 1:
        print(f"{step:7d} | {energy.item():14.6f} | {current_a:.6f}")

print(f"\n最終エネルギー: {energy_joint_history[-1]:.6f} eV")
print(f"最終格子定数: a = {lattice_a_joint_history[-1]:.6f} Å")

In [ ]:
# 同時最適化の履歴をプロット
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

# 左軸: エネルギー
ax.plot(energy_joint_history, 'b-', linewidth=2, label='エネルギー')
ax.set_xlabel('最適化ステップ', fontsize=12)
ax.set_ylabel('エネルギー (eV)', fontsize=12, color='b')
ax.tick_params(axis='y', labelcolor='b')
ax.grid(True, alpha=0.3)

# 右軸: 格子定数
ax2 = ax.twinx()
ax2.plot(lattice_a_joint_history, 'g-', linewidth=2, label='格子定数 a')
ax2.axhline(y=5.43, color='r', linestyle='--', linewidth=2, label='正解値')
ax2.set_ylabel('格子定数 a (Å)', fontsize=12, color='g')
ax2.tick_params(axis='y', labelcolor='g')

# タイトルと凡例
ax.set_title('座標と格子の同時最適化', fontsize=14)
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()

✅ **結果**: 座標と格子の両方が同時に最適化されました！

## 5. まとめ

このノートブックでは、以下を実演しました：

1. ✅ **勾配の計算**: 座標に対する勾配を backward() で計算
2. ✅ **構造最適化**: ランダムに摂動した構造を Adam で最適化
3. ✅ **格子最適化**: 格子定数を SGD で最適化
4. ✅ **同時最適化**: 座標と格子を同時に最適化

### 主なポイント

- `DifferentiableMatterSimCalculator` を使用
- 入力 tensor に `requires_grad=True` を設定
- `energy.backward()` で勾配を計算
- PyTorch の optimizer で入力を更新

### 次のステップ

- より複雑な損失関数を定義（例: 目標構造との距離）
- 複数構造のバッチ処理
- カスタム最適化アルゴリズムの実装

詳細なドキュメントは `docs/DIFFERENTIABLE_API.md` を参照してください。